In [7]:
import psycopg2
import pandas as pd

conexao = psycopg2.connect(host="kesavan.db.elephantsql.com", database="cjwdogrw", user="cjwdogrw", password="acihzKCpPs16aX1MGNmlYqY1OV--19Q6")
cursor = conexao.cursor()

In [2]:
cursor.execute("""select cod_ped_prod, preco from pedido_produto 
                  where preco < (select avg(pedido_produto.preco) from pedido_produto);""")

data = cursor.fetchall()

colunas = []

for col in cursor.description:
  colunas.append(col[0])

select1 = pd.DataFrame(data=data,columns = colunas) 

select1

,cod_ped_prod,preco
0,4,30.0
1,5,20.0
2,8,20.0


In [3]:
cursor.execute("""SELECT 	ingrediente.nome AS nome_ingrediente, 
                          tipo_medida.descricao AS tipo_medida
                  FROM ingrediente
                  INNER JOIN tipo_medida ON (ingrediente.fk_cod_tipo_medida = tipo_medida.cod_tipo_medida) 
                  WHERE tipo_medida.descricao NOT IN (SELECT tipo_medida.descricao FROM tipo_medida WHERE tipo_medida.descricao = 'Unidade' OR tipo_medida.descricao = 'Colher de sopa'); """)

data = cursor.fetchall()

colunas = []

for col in cursor.description:
  colunas.append(col[0])

select2 = pd.DataFrame(data=data,columns = colunas) 

select2

,nome,descricao
0,Açucar,Xicara
1,Fubá,Xicara
2,Óleo,Copo
3,Leite,Copo
4,Leite condesado,Caixa
5,Chocolate granulado,A gosto


In [8]:
cursor.execute("""SELECT 	produto.cod_produto,
                      sum(pedido.preco_total) AS preco_total_lucrado
                  FROM pedido 
                  INNER JOIN pedido_produto ON (pedido.cod_pedido = pedido_produto.fk_cod_pedido)
                  INNER JOIN produto ON (pedido_produto.fk_cod_produto = produto.cod_produto)
                  WHERE pedido.preco_total IN (SELECT pedido.preco_total FROM pedido WHERE pedido.preco_total > 50) 
                  GROUP BY produto.cod_produto
                  ORDER BY produto.cod_produto""")

data = cursor.fetchall()

colunas = []

for col in cursor.description:
  colunas.append(col[0])

select3 = pd.DataFrame(data=data,columns = colunas) 

select3

,cod_produto,preco_total_lucrado
0,1,250.0
1,3,250.0
2,5,100.0


In [9]:
cursor.execute("""select pedido.cod_pedido, pedido.preco_total, produto.preco 
                  from pedido 
                  inner join (select distinct produto.preco from pedido_produto
                              inner join produto on (pedido_produto.fk_cod_produto = produto.cod_produto)) as produto
                  on (pedido.preco_total = produto.preco); """)

data = cursor.fetchall()

colunas = []

for col in cursor.description:
  colunas.append(col[0])

select4 = pd.DataFrame(data=data,columns = colunas) 

select4

,cod_pedido,preco_total,preco
0,2,50.0,50.0
1,5,50.0,50.0


In [10]:
cursor.execute("""select (count(fk_cod_ingrediente)) as ing_por_produto
                  from produto_ingrediente 
                  group by fk_cod_produto
                  having count(fk_cod_ingrediente) < (
                    select (count(fk_cod_ingrediente))
                    from produto_ingrediente 
                    group by fk_cod_produto
                    limit(1)) """)

data = cursor.fetchall()

colunas = []

for col in cursor.description:
  colunas.append(col[0])

select5 = pd.DataFrame(data=data,columns = colunas) 

select5

,ing_por_produto
0,4


In [ ]:
conexao.close();